In [ ]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
#import scikitplot as skplt

from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report,confusion_matrix

from sklearn.utils.class_weight import compute_sample_weight

In [ ]:
from sklearn.model_selection import StratifiedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn import metrics

from sklearn.model_selection import GridSearchCV
import pickle

In [ ]:
X_tr_DCR = pd.read_csv('data/classification/DCR/X_train.csv', index_col=0)
X_t_DCR = pd.read_csv('data/classification/DCR/X_val.csv', index_col=0)
y_tr_DCR = pd.read_csv('data/classification/DCR/y_train.csv', index_col=0)
y_t_DCR = pd.read_csv('data/classification/DCR/y_val.csv', index_col=0)


X_tr_OS = pd.read_csv('data/survival/OS/X_train.csv', index_col=0)
X_t_OS = pd.read_csv('data/survival/OS/X_val.csv', index_col=0)
y_tr_OS = pd.read_csv('data/survival/OS/y_train.csv', index_col=0)
y_t_OS = pd.read_csv('data/survival/OS/y_val.csv', index_col=0)

X_tr_PFS = pd.read_csv('data/survival/PFS/X_train.csv', index_col=0)
X_t_PFS = pd.read_csv('data/survival/PFS/X_val.csv', index_col=0)
y_tr_PFS = pd.read_csv('data/survival/PFS/y_train.csv', index_col=0)
y_t_PFS = pd.read_csv('data/survival/PFS/y_val.csv', index_col=0)


# values of the event must be boolean
y_tr_OS['STATUS OS']=y_tr_OS['STATUS OS'].astype(bool)
y_t_OS['STATUS OS']=y_t_OS['STATUS OS'].astype(bool)
y_tr_PFS['STATUS PD']=y_tr_PFS['STATUS PD'].astype(bool)
y_t_PFS['STATUS PD']=y_t_PFS['STATUS PD'].astype(bool)
# targets must be an array (not a dataframe)
y_tr_OS=y_tr_OS.to_records(index=False)
y_t_OS=y_t_OS.to_records(index=False)
y_tr_PFS=y_tr_PFS.to_records(index=False)
y_t_PFS=y_t_PFS.to_records(index=False)

from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True)


In [ ]:
sw_train = compute_sample_weight(class_weight='balanced', y=y_tr_DCR)
sw_test = compute_sample_weight(class_weight='balanced', y=y_t_DCR)

# DCR

In [ ]:
# import best classification model
path = 'results/classification/DCR/RFE/MODELS/RF.pkl'
path

In [ ]:
mod = pickle.load(open(path, 'rb'))

In [ ]:
feats = mod.feature_names_in_

In [ ]:
# CLINICAL FEATURES TO REMOVE
drop_clin = ['Bone metastases', 'logNLR', 'Heng Score at baseline']  # RF - RFE

# TOP 3 SHAP FEATURES
drop_shap = ['logPLR', 'logSII',"logEosinophils"]

In [ ]:
X_train = X_tr_DCR.copy()
X_test = X_t_DCR.copy()

feats = mod.feature_names_in_

#   with LGBM
#feats = mod.feature_name_
#feats = [feats[i].replace('_',' ') for i in range(len(feats))]

X_train = X_train.loc[:,feats]
X_test = X_test.loc[:,feats]
X_train.shape

In [ ]:
params = [
    {'max_depth': [2], 'min_samples_leaf': [4], 'min_samples_split': [2], 'n_estimators': [50]}
]

model = RandomForestClassifier()

models = [ ]

X_tr = [
    X_train,
    X_train.copy().drop(drop_shap, axis = 1),
    X_train.copy().drop(drop_clin, axis = 1)
]

X_t = [
    X_test,
    X_test.copy().drop(drop_shap, axis = 1),
    X_test.copy().drop(drop_clin, axis = 1)
]

y_train = y_tr_DCR.copy()
y_test = y_t_DCR.copy()

In [ ]:
import statistics 

F1_train = []
F1_test = []
fit_params = {'sample_weight': sw_train}

f1_tr = []
f1_t = []

# AVERAGE 10 TIMES THE MODEL WITH SELECTED FEATURES
for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[0], y_train, **fit_params)
    new_m1 = gs1.best_estimator_
    new_m1.fit(X_tr[0], y_train,**fit_params)
    
    f1_tr.append(f1_score(y_train, new_m1.predict(X_tr[0]), average="weighted", sample_weight=sw_train))
    f1_t.append(f1_score(y_test, new_m1.predict(X_t[0]), average="weighted", sample_weight=sw_test))
    

F1_train.append(statistics.mean(f1_tr))
F1_test.append(statistics.mean(f1_t))

models.append(new_m1)


f1_tr = []
f1_t = []

del gs1
# AVERAGE 10 TIMES THE MODEL WITHOUT SHAP FEATURES
for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[1], y_train, **fit_params)
    new_m1 = gs1.best_estimator_
    new_m1.fit(X_tr[1], y_train,**fit_params)
    
    f1_tr.append(f1_score(y_train, new_m1.predict(X_tr[1]), average="weighted", sample_weight=sw_train))
    f1_t.append(f1_score(y_test, new_m1.predict(X_t[1]), average="weighted", sample_weight=sw_test))
    

F1_train.append(statistics.mean(f1_tr))
F1_test.append(statistics.mean(f1_t))

models.append(new_m1)


f1_tr = []
f1_t = []

del gs1
# AVERAGE 10 TIMES THE MODEL WITHOUT CLINICAL FEATURES
for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[2], y_train, **fit_params)
    new_m1 = gs1.best_estimator_
    new_m1.fit(X_tr[2], y_train,**fit_params)
    
    f1_tr.append(f1_score(y_train, new_m1.predict(X_tr[2]), average="weighted", sample_weight=sw_train))
    f1_t.append(f1_score(y_test, new_m1.predict(X_t[2]), average="weighted", sample_weight=sw_test))
    

F1_train.append(statistics.mean(f1_tr))
F1_test.append(statistics.mean(f1_t))


models.append(new_m1)


In [ ]:
print(F1_train)
print(F1_test)

### BOOTSTRAPPING

In [ ]:
n_iter = 1000
import random, math
results = []

for i in range(0,3):
    result = []
    for j in range(n_iter):
        x = [random.randint(0,X_t[i].shape[0]-1) for k in range(0,math.floor(X_t[i].shape[0]/2))]
        test = X_t[i].copy()
        test = test.iloc[x,:]
        y_t_bs = y_test.copy()
        y_t_bs = y_t_bs.iloc[x,:]
        sw_x = compute_sample_weight(class_weight='balanced', y=y_t_bs)
        res = f1_score(y_t_bs, models[i].predict(test), average="weighted", sample_weight=sw_x)
        result.append(res)

    result.sort()
    results.append(result)


In [ ]:
# confidence intervals
alpha = 0.95
p_low = ((1.0-alpha)/2.0) * 100
p_high = (alpha+((1.0-alpha)/2.0)) * 100

for i in range(0,3):
    lower = max(0.0, np.percentile(results[i], p_low))
    upper = min(1.0, np.percentile(results[i], p_high))
    print('%.0f%% confidence interval %.3f and %.3f' % (alpha*100, lower, upper))

# OS

In [ ]:

from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_censored, cumulative_dynamic_auc, concordance_index_ipcw, integrated_brier_score
from sksurv.ensemble import RandomSurvivalForest, ExtraSurvivalTrees
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.svm import FastKernelSurvivalSVM

In [ ]:
path = 'results/survival/OS/MODELS(ALL)/EST.pkl'  # EST-ALL
path

In [ ]:
mod = pickle.load(open(path, 'rb'))
mod

In [ ]:
feats = mod.feature_names_in_
feats

In [ ]:
X_train = X_tr_OS.copy()
X_test = X_t_OS.copy()

X_train = X_train.loc[:,feats]
X_test = X_test.loc[:,feats]

In [ ]:
drop_clin = ['Bone metastases', 'logNLR', 'Heng Score at baseline']

drop_shap = ['KPS',"Surgery", 'logPLR']  
models=[]

In [ ]:
params = {'max_depth': [3], 'min_samples_leaf': [3], 'min_samples_split': [3], 'n_estimators': [100]}

model = ExtraSurvivalTrees()

X_tr = [
    X_train,
    X_train.copy().drop(drop_shap, axis = 1),
    X_train.copy().drop(drop_clin, axis = 1)
]

X_t = [
    X_test,
    X_test.copy().drop(drop_shap, axis = 1),
    X_test.copy().drop(drop_clin, axis = 1)
]

y_tr = y_tr_OS.copy()
y_t = y_t_OS.copy()

In [ ]:
import statistics
ci_train=[ ]
ci_test=[  ]


CIT = []
CITR = []

for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[0], y_tr)
    new_m1 = gs1.best_estimator_
    ci_tr = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], new_m1.predict(X_tr[0]))
    ci_t = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], new_m1.predict(X_t[0]))
    CITR.append(ci_tr[0])
    CIT.append(ci_t[0])

ci_train.append(statistics.mean(CITR))
ci_test.append(statistics.mean(CIT))

models.append(new_m1)

CIT = []
CITR = []

for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[1], y_tr)
    new_m1 = gs1.best_estimator_
    ci_tr = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], new_m1.predict(X_tr[1]))
    ci_t = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], new_m1.predict(X_t[1]))
    CITR.append(ci_tr[0])
    CIT.append(ci_t[0])

ci_train.append(statistics.mean(CITR))
ci_test.append(statistics.mean(CIT))
    
models.append(new_m1)

CIT = []
CITR = []

del gs1

for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[2], y_tr)
    new_m1 = gs1.best_estimator_
    ci_tr = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], new_m1.predict(X_tr[2]))
    ci_t = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], new_m1.predict(X_t[2]))
    CITR.append(ci_tr[0])
    CIT.append(ci_t[0])

ci_train.append(statistics.mean(CITR))
ci_test.append(statistics.mean(CIT))

models.append(new_m1)



In [ ]:
print(ci_train)
print(ci_test)

### BOOTSTRAPPING

In [ ]:
n_iter = 1000
import random, math
results = []

for i in range(0,3):
    result = []
    for j in range(n_iter):
        x = [random.randint(0,X_t[i].shape[0]-1) for k in range(0,math.floor(X_t[i].shape[0]/2))]
        test = X_t[i].copy()
        test = test.iloc[x,:]
        y_t_bs = y_t.copy()
        y_t_bs = y_t_bs[x]
        res = concordance_index_censored(y_t_bs["STATUS OS"], y_t_bs["OS"], models[i].predict(test))
        result.append(res[0])

    result.sort()
    results.append(result)


In [ ]:
# confidence intervals
alpha = 0.95
p_low = ((1.0-alpha)/2.0) * 100
p_high = (alpha+((1.0-alpha)/2.0)) * 100

for i in range(0,3):
    lower = max(0.0, np.percentile(results[i], p_low))
    upper = min(1.0, np.percentile(results[i], p_high))
    print('%.0f%% confidence interval %.3f and %.3f' % (alpha*100, lower, upper))


# PFS

In [ ]:
path = 'results/survival/PFS/MODELS(ALL)/GB.pkl'
path

In [ ]:
mod = pickle.load(open(path, 'rb'))


In [ ]:
feats = mod.feature_names_in_
feats

In [ ]:
X_train = X_tr_PFS.copy()
X_test = X_t_PFS.copy()

X_train = X_train.loc[:,feats]
X_test = X_test.loc[:,feats]

In [ ]:
drop_clin = ['Bone metastases', 'logNLR', 'Heng Score at baseline']  

drop_shap =  ['KPS',"logEosinophils", 'logPLR']

In [ ]:
params = {'learning_rate': [0.1], 'max_depth': [2], 'min_samples_leaf': [4], 'min_samples_split': [3]}


model = GradientBoostingSurvivalAnalysis()


models = []

X_tr = [
    X_train,
    X_train.copy().drop(drop_shap, axis = 1),
    X_train.copy().drop(drop_clin, axis = 1)
]

X_t = [
    X_test,
    X_test.copy().drop(drop_shap, axis = 1),
    X_test.copy().drop(drop_clin, axis = 1)
]

y_tr = y_tr_PFS.copy()
y_t = y_t_PFS.copy()

In [ ]:
import statistics
ci_train=[ ]
ci_test=[  ]


CIT = []
CITR = []

for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[0], y_tr)
    new_m1 = gs1.best_estimator_
    ci_tr = concordance_index_censored(y_tr["STATUS PD"], y_tr["PFS"], new_m1.predict(X_tr[0]))
    ci_t = concordance_index_censored(y_t["STATUS PD"], y_t["PFS"], new_m1.predict(X_t[0]))
    CITR.append(ci_tr[0])
    CIT.append(ci_t[0])

ci_train.append(statistics.mean(CITR))
ci_test.append(statistics.mean(CIT))

models.append(new_m1)

CIT = []
CITR = []

for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[1], y_tr)
    new_m1 = gs1.best_estimator_
    ci_tr = concordance_index_censored(y_tr["STATUS PD"], y_tr["PFS"], new_m1.predict(X_tr[1]))
    ci_t = concordance_index_censored(y_t["STATUS PD"], y_t["PFS"], new_m1.predict(X_t[1]))
    CITR.append(ci_tr[0])
    CIT.append(ci_t[0])

ci_train.append(statistics.mean(CITR))
ci_test.append(statistics.mean(CIT))
    
models.append(new_m1)

CIT = []
CITR = []

del gs1

for i in range(10):
    gs1 = GridSearchCV(model,params,cv=cv,return_train_score=True, n_jobs=-1)
    gs1.fit(X_tr[2], y_tr)
    new_m1 = gs1.best_estimator_
    ci_tr = concordance_index_censored(y_tr["STATUS PD"], y_tr["PFS"], new_m1.predict(X_tr[2]))
    ci_t = concordance_index_censored(y_t["STATUS PD"], y_t["PFS"], new_m1.predict(X_t[2]))
    CITR.append(ci_tr[0])
    CIT.append(ci_t[0])

ci_train.append(statistics.mean(CITR))
ci_test.append(statistics.mean(CIT))

models.append(new_m1)



In [ ]:
print(ci_train)
print(ci_test)

### BOOTSTRAPPING

In [ ]:
n_iter = 1000
import random, math
results = []

for i in range(0,3):
    result = []
    for j in range(n_iter):
        x = [random.randint(0,X_t[i].shape[0]-1) for k in range(0,math.floor(X_t[i].shape[0]/2))]
        test = X_t[i].copy()
        test = test.iloc[x,:]
        y_t_bs = y_t.copy()
        y_t_bs = y_t_bs[x]
        res = concordance_index_censored(y_t_bs["STATUS PD"], y_t_bs["PFS"], models[i].predict(test))
        result.append(res[0])

    result.sort()
    results.append(result)


In [ ]:
# confidence intervals
alpha = 0.95
p_low = ((1.0-alpha)/2.0) * 100
p_high = (alpha+((1.0-alpha)/2.0)) * 100

for i in range(0,3):
    lower = max(0.0, np.percentile(results[i], p_low))
    upper = min(1.0, np.percentile(results[i], p_high))
    print('%.0f%% confidence interval %.3f and %.3f' % (alpha*100, lower, upper))